In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn.metrics
from sklearn.multiclass import OneVsRestClassifier

In [18]:
data = pd.read_excel("C:/Users/Donát/Documents/GitHub/-nkutfel/2019/2019_zh_felv_jegy_only.xlsx")

In [19]:
#szűrjük ki a nem kategorikus vagy numerikus változókat
working_data = data.drop(labels = ["Név","Születési dátum", "Képzés"], axis = 1)

In [20]:
working_data

,Érettségi éve,Tanulmányi pont,Érettségi pont,Többletpont,Törzs pont,Összes pont,ZH0,Matematika A1a - Analízis,Matematika A2c
0,2019,197,195,100,392,492,51,3,3
1,2019,196,197,90,394,484,55,5,5
2,2019,192,186,100,378,478,34,3,2
3,2019,193,192,90,385,475,37,3,2
4,2019,192,186,93,378,471,31,3,2
...,...,...,...,...,...,...,...,...,...
265,2019,186,157,78,343,421,10,2,2
266,2019,187,142,90,329,419,24,4,4
267,2019,169,170,78,340,418,24,2,0
268,2019,182,146,90,328,418,21,2,3


In [21]:
#szedjük ki az Összes pontot és a Törzspontot is, hisz az valójábon három attribútum értékének az összege
working_data = working_data.drop(labels = ["Összes pont", "Törzs pont"], axis = 1)

In [22]:
features = working_data.drop(labels = ["Matematika A1a - Analízis", "Matematika A2c"], axis = 1) #az első évi jegyre
label = working_data["Matematika A1a - Analízis"]
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.3, random_state=0)

In [23]:
#SVM-re
from sklearn.svm import SVC
clf = OneVsRestClassifier(SVC(class_weight = "balanced", random_state=0)).fit(X_train, y_train)     # balanced nélkül romlik a pontosság
y_pred = clf.predict(X_test)

print(f'Accuracy: {round(sklearn.metrics.accuracy_score(y_test, y_pred),2)}')
print(f'Precision: {round(sklearn.metrics.precision_score(y_test, y_pred, average = "macro"),2)}')
print(f'Recall: {round(sklearn.metrics.recall_score(y_test, y_pred, average = "macro"),2)}')


#how

Accuracy: 0.02
Precision: 0.0
Recall: 0.2


D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
#logisztikus regresszióra
from sklearn.linear_model import LogisticRegression

logreg = OneVsRestClassifier(LogisticRegression(random_state=0, max_iter = 1000)).fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print(f'Accuracy: {round(sklearn.metrics.accuracy_score(y_test, y_pred),2)}')
print(f'Precision: {round(sklearn.metrics.precision_score(y_test, y_pred, average = "macro"),2)}')
print(f'Recall: {round(sklearn.metrics.recall_score(y_test, y_pred, average = "macro"),2)}')

Accuracy: 0.33
Precision: 0.23
Recall: 0.25


D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
#ezek után nem oneVSall: lineáris regresszió
from sklearn.linear_model import LinearRegression

reg_alap = LinearRegression().fit(features,label)
reg_alap.score(features, label)                        #nem túl magas

0.28714568627339154

In [27]:
#most nézzük a tanítóhalmazon és a teszthalmazon

reg = LinearRegression().fit(X_train,y_train)
reg_pred = reg.predict(X_test)

#együtthatók
print("Coefficients:", reg.coef_)
#RMSE

rmse = np.power(np.sum(np.power(y_pred - reg_pred, 2))/reg_pred.shape[0],0.5)
print("RMSE:", rmse)
#nem olyan rossz

Coefficients: [-0.12873261 -0.00152868  0.00697631 -0.00393876  0.03113221]
RMSE: 0.7197768154072768


In [28]:
#R^2 és SSR
SST = np.sum(np.power(y_pred-np.mean(y_pred),2))
SSR = np.sum(np.power(reg_pred-np.mean(y_pred),2))
R2 = SSR/SST
R2

0.3660659446052894

In [29]:
working_data

,Érettségi éve,Tanulmányi pont,Érettségi pont,Többletpont,ZH0,Matematika A1a - Analízis,Matematika A2c
0,2019,197,195,100,51,3,3
1,2019,196,197,90,55,5,5
2,2019,192,186,100,34,3,2
3,2019,193,192,90,37,3,2
4,2019,192,186,93,31,3,2
...,...,...,...,...,...,...,...
265,2019,186,157,78,10,2,2
266,2019,187,142,90,24,4,4
267,2019,169,170,78,24,2,0
268,2019,182,146,90,21,2,3


In [34]:
#másik: kerekítsük a becsült értékeket

for i in range(reg_pred.shape[0]):
    reg_pred[i] = round(reg_pred[i],0)

In [35]:
#R^2 és SSR
SST = np.sum(np.power(y_pred-np.mean(y_pred),2))
SSR = np.sum(np.power(reg_pred-np.mean(y_pred),2))
R2 = SSR/SST
R2

0.5246380846325167